## Load required modules

In [ ]:
import matplotlib.pyplot as plt
import json
import pickle
import numpy as np
import pprint as pp

from tensorflow.keras.models import model_from_json

## User settings

In [ ]:
application_root_directory = '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forecasting/deep_learning'
training_run = 'blah----71807266-3119-4e24-b788-f97511b82567'
data_source_root_directory = '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forecasting/deep_learning/pipeline_components/output/queries'
data_source_id = '309457bc-a227-4332-8c0b-2cf5dd38749c'

## Initialize

In [ ]:
model_training_output_directory = application_root_directory + '/output/'

filename_config_json = model_training_output_directory + '/' + training_run + '_lstm_regressor_config.json'
filename_model_json = model_training_output_directory + '/' + training_run + '_model_regressor.json'
filename_model_final_weights = model_training_output_directory + '/' + training_run + '_final_weights_regressor.pickled'
filename_history = model_training_output_directory + '/' + training_run + '_final_history_regressor.pickled'

filename_train_val_test_data = data_source_root_directory + '/full_train_val_test_' + data_source_id + '.pickled'

## Define function to load the configuration used for training the model

In [ ]:
def load_training_run_config(filename_config_json):
    with open(filename_config_json) as fff:
        config = json.load(fff)
    return config

## Define function to load the trained model

In [ ]:
def load_model(filename_model_json, filename_model_final_weights):
    with open(filename_model_json, 'r') as json_file:
        loaded_model_json = json_file.read()

    # Create a new model from the JSON
    model = model_from_json(loaded_model_json)

    # Load the weights into the new model
    model.load_weights(filename_model_final_weights)

    return model

## Define function to load the training history

In [ ]:
def load_training_history(filename_history):
    try:
        with open(filename_history, 'rb') as fff:
            history = pickle.load(fff)
    except Exception as exception:
        print(exception)
        sys.exit(-1)
    return history  

## Define function to plot training loss function

In [ ]:
def plot_basic_loss(history, config, metric_base = 'loss', ylabel = 'Loss'):
    epochs = range(1, len(history[metric_base]) + 1)

    plt.figure()
    plt.plot(epochs, history[metric_base], '-.', label = ylabel)
    plt.plot(epochs, history['val_' + metric_base], '-.', label = 'Validation ' + ylabel)
    plt.xlabel('Training Epoch')
    
    plt.ylabel(ylabel)
    plt.title('Training Run: ' + training_run.split('----')[-1] + '\nBatch Size = ' + str(config['batch_size']))
    
    plt.legend()
    plt.tight_layout()
    plt.show()
    plt.close()

## Define function to load the training, validation and testing sets

In [ ]:
def load_train_val_test_data(filename_train_val_test):
    with open(filename_train_val_test_data, 'rb') as fff:
        train_val_test_dict = pickle.load(fff)
    return train_val_test_dict

## Define function to predict results using the model

In [ ]:
def predict(X, model):
    y_predicted = model.predict(X)
    return y_predicted

## Define function to plot a given forecast

In [ ]:
def plot_a_forecast(X, y_known, y_predicted, y_forward, model, index = 20, predictor_line = 0):

    indices = np.arange(0, X[index].shape[0])
    indices_forward = np.arange(len(indices), len(indices) + len(y_forward[index, :]))

    X_mean = np.mean(X[index, :, predictor_line])

    y_predicted_min = y_predicted[index, 0]
    y_predicted_mean = y_predicted[index, 1]  # check index of n = 1
    y_predicted_median = y_predicted[index, 2]  # check index of n = 1
    y_predicted_max = y_predicted[index, 3]
    
    plt.figure()
    plt.plot(indices, X[index, :, predictor_line])
    plt.plot(indices, [X_mean] * len(indices))
    
    plt.plot(indices_forward, y_forward[index, :])
    plt.plot(indices_forward, [y_predicted_min] * len(indices_forward))
    plt.plot(indices_forward, [y_predicted_mean] * len(indices_forward))
    plt.plot(indices_forward, [y_predicted_median] * len(indices_forward))
    plt.plot(indices_forward, [y_predicted_max] * len(indices_forward))
    
    plt.show()
    plt.close()

## Load and display the training configuration file

In [ ]:
config = load_training_run_config(filename_config_json)

In [ ]:
pp.pprint(config)

## Load the trained model

In [ ]:
model = load_model(filename_model_json, filename_model_final_weights)

## Load and plot the training history

In [ ]:
history = load_training_history(filename_history)

In [ ]:
plot_basic_loss(history, config)

## Load the training, validation and testing data

In [ ]:
train_val_test_dict = load_train_val_test_data(filename_train_val_test_data)

## Predict the y values for the test set

In [ ]:
y_predicted = predict(train_val_test_dict['test']['X'], model)

## Plot a forecast

In [ ]:
plot_a_forecast(
    train_val_test_dict['test']['X'],
    train_val_test_dict['test'][config['y_set']],
    y_predicted,
    train_val_test_dict['test'][config['y_forward_set']],
    model,
)